In [1]:
import random
from operator import attrgetter
from collections import Counter

try:
    del Counter.__reduce__
except: pass

import numpy


from deap import algorithms
from deap import base
from deap import creator
from deap import tools


In [3]:
ITEMS = {
    'AKP':(42.5,37),
    'CHP':(27.1,23,4),
    'İYİ':(16.7,15,7),
    'HDP':(11.0,14,1),
    'SAADET':(2.5,3)}

ITEMS_NAME =list(ITEMS.keys())

In [4]:
creator.create("FitnessMulti",base.Fitness,weights=(1.0,-1.0,1.0))

creator.create("Individual",Counter,fitness=creator.FitnessMulti)


IND_INIT_SIZE=2

toolbox=base.Toolbox()

toolbox.register("attr_item", random.choice, ITEMS_NAME)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_item, IND_INIT_SIZE)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


In [5]:
def evalXKCD(individual, target_rate):
    milletvekili = 0.0
    sandik= [0]
    oy= 0
    for item, number in individual.items():
        if number > 0:
            milletvekili += ITEMS[item][0] * number
            sandik.append(ITEMS[item][1])
            oy += number
    return (milletvekili - target_rate), max(sandik), oy

In [7]:
evalXKCD(creator.Individual({'AKP':1,
    'CHP':1,
    'İYİ':0,
    'HDP':1,
    'SAADET':0}),51.0)

(29.599999999999994, 37, 3)

In [8]:
def cxCounter(ind1, ind2, indpb):
    """Swaps the number of randomly-chosen items between two individuals"""
    for key in ITEMS.keys():
        if random.random() < indpb:
            ind1[key], ind2[key] = ind2[key], ind1[key]
    return ind1, ind2

In [9]:
cxCounter(creator.Individual({'AKP':0,
    'CHP':0,
    'İYİ':0,
    'HDP':0,
    'SAADET':0}),
    creator.Individual({'AKP':0,
    'CHP':0,
    'İYİ':0,
    'HDP':0,
    'SAADET':0}),0.50)
    


(Individual({'AKP': 0,
             'CHP': 0,
             'HDP': 0,
             'SAADET': 0,
             '\xc4\xb0Y\xc4\xb0': 0}),
 Individual({'AKP': 0,
             'CHP': 0,
             'HDP': 0,
             'SAADET': 0,
             '\xc4\xb0Y\xc4\xb0': 0}))

In [10]:
def mutCounter(individual):
    """Adds or remove an item from an individual"""
    if random.random() > 0.5:
        individual.update([random.choice(ITEMS_NAME)])
    else:
        val = random.choice(ITEMS_NAME)
        individual.subtract([val])
        if individual[val] < 0:
            del individual[val]
    return individual,

In [12]:
mutCounter(creator.Individual({'AKP':1,
    'CHP':1,
    'İYİ':0,
    'HDP':1,
    'SAADET':0}))

(Individual({'AKP': 1,
             'CHP': 0,
             'HDP': 1,
             'SAADET': 0,
             '\xc4\xb0Y\xc4\xb0': 0}),)

In [13]:
import sys

toolbox.register("evaluate", evalXKCD, target_rate=51.0)


toolbox.decorate("evaluate", tools.DeltaPenalty(lambda ind: evalXKCD(ind, 15.05)[0] <= 0,
                                               (-sys.float_info.max,
                                                sys.float_info.max,
                                                -sys.float_info.max)))
toolbox.register("mate", cxCounter, indpb=0.5)
toolbox.register("mutate", mutCounter)
toolbox.register("select", tools.selNSGA2)

In [14]:
NGEN=100

MU=50

LAMBDA=20

CXPB=0.3

MUTPB=0.3

pop=toolbox.population(n=MU)

In [15]:
pop

[Individual({'HDP': 1, 'SAADET': 1}),
 Individual({'\xc4\xb0Y\xc4\xb0': 2}),
 Individual({'\xc4\xb0Y\xc4\xb0': 2}),
 Individual({'CHP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'HDP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'CHP': 2}),
 Individual({'HDP': 1, 'SAADET': 1}),
 Individual({'AKP': 1, 'HDP': 1}),
 Individual({'HDP': 2}),
 Individual({'CHP': 1, 'HDP': 1}),
 Individual({'CHP': 1, 'SAADET': 1}),
 Individual({'AKP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'CHP': 1, 'HDP': 1}),
 Individual({'AKP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'CHP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'HDP': 1, 'SAADET': 1}),
 Individual({'AKP': 1, 'HDP': 1}),
 Individual({'CHP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'CHP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'CHP': 1, 'HDP': 1}),
 Individual({'HDP': 1, 'SAADET': 1}),
 Individual({'AKP': 1, 'CHP': 1}),
 Individual({'AKP': 2}),
 Individual({'CHP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Individual({'HDP': 1, '\xc4\xb0Y\xc4\xb0': 1}),
 Indiv

In [16]:
hof=tools.ParetoFront()

milletvekili_stats = tools.Statistics(key=lambda ind: ind.fitness.values[0])
sandik_stats = tools.Statistics(key=lambda ind: ind.fitness.values[1])
oy_stats = tools.Statistics(key=lambda ind: ind.fitness.values[2])

stats = tools.MultiStatistics(milletvekili=milletvekili_stats,sandik=sandik_stats, oy=oy_stats)
stats.register("avg", numpy.mean, axis=0)

algorithms.eaMuPlusLambda(pop, toolbox, MU, LAMBDA, CXPB, MUTPB, NGEN,
                          stats, halloffame=hof)

   	      	milletvekili	 oy 	sandik
   	      	----        	----	---   
gen	nevals	avg         	avg 	avg   
0  	50    	-inf        	-inf	inf   
1  	15    	-inf        	-inf	inf   
2  	13    	-inf        	-inf	inf   
3  	13    	-inf        	-inf	inf   
4  	9     	-inf        	-inf	inf   
5  	14    	-inf        	-inf	inf   
6  	9     	-40.45      	1.84	10.46 
7  	9     	-41.91      	1.8 	8.64  
8  	9     	-43.64      	1.72	6.54  
9  	14    	-42.52      	1.76	7.86  
10 	7     	-41.7       	1.68	9.06  
11 	12    	-40.71      	1.6 	10.48 
12 	8     	-40.98      	1.56	10.2  
13 	9     	-42.06      	1.4 	9.08  
14 	9     	-43.51      	1.16	7.62  
15 	11    	-41.22      	1.6 	9.7   
16 	12    	-42.64      	1.44	8.08  
17 	11    	-40.48      	1.76	10.32 
18 	9     	-41.16      	1.76	9.32  
19 	13    	-41.91      	1.8 	8.1   
20 	10    	-40.02      	2.08	10.06 
21 	9     	-40.97      	2.04	8.78  
22 	15    	-42.33      	2.04	6.78  
23 	14    	-44.23      	1.96	4.22  
24 	11    	-44.58      	1.48

([Individual({'CHP': 0, 'HDP': 0, 'SAADET': 6, '\xc4\xb0Y\xc4\xb0': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HDP': 0}),
  Individual({'CHP': 0, 'HD

In [17]:
for i in hof:
    print(i, i.fitness.values)

(Individual({'SAADET': 6, 'AKP': 0, '\xc4\xb0Y\xc4\xb0': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, 'HDP': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, 'AKP': 0, '\xc4\xb0Y\xc4\xb0': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, 'AKP': 0, '\xc4\xb0Y\xc4\xb0': 0, 'HDP': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'AKP': 0, 'HDP': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, 'AKP': 0, 'HDP': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, '\xc4\xb0Y\xc4\xb0': 0, 'HDP': 0}), (-36.0, 3.0, 6.0))
(Individual({'SAADET': 6, 'CHP': 0, '\xc4\xb0Y\xc4\xb0': 0}), (-36.0, 3.0, 6.0))
(Individual({'AKP': 0, '\xc4\xb0Y\xc4\xb0': 0, 'HDP': 0}), (-51.0, 0.0, 0.0))
(Individual({'SAADET': 0, 'AKP': 0, 'HDP': 0}), (-51.0, 0.0, 0.0))
(Individual({'CHP': 0}), (-51.0, 0.0, 0.0))
(Individual({'\xc4\xb0Y\xc4\xb0': 0, 'HDP': 0}), (-51.0, 0.0, 0.0))
(Individual({'CHP': 0, 'AKP': 0, 'HDP': 0}), (-51.0, 0.0, 0.0))
(Individual({'CHP': 0, 

In [18]:
print("Best:", hof[0], hof[0].fitness.values)


('Best:', Individual({'SAADET': 6, 'AKP': 0, '\xc4\xb0Y\xc4\xb0': 0}), (-36.0, 3.0, 6.0))
